In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from src.dysregnet.dysregnet import run

In [2]:
meta = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/LUAD_meta.csv")
expr = pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/LUAD_expression.csv")
grn=pd.read_csv("/Users/juliakuhn/Desktop/uni/WS2425/Systems_BioMedicine/Project_Phase/exercise_data/regulatory_network.csv", sep = ",")

In [3]:
meta["birth_days_to"] = meta["birth_days_to"].fillna(meta["birth_days_to"].mean())
meta["race"] = meta["race"].fillna("not reported")
meta["race"] = meta["race"].replace({"[Unknown]": "not reported", "[Not Evaluated]":"not reported"})

expr = expr.set_index(expr.columns[0])
expr = expr.T
expr.insert(0, "sample", expr.index)
assert all(expr.iloc[:, 0].values == meta.iloc[:,0].values)

In [4]:
meta.head()

,sample,_PATIENT,cancer type abbreviation,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction,sample_type_id,sample_type,_primary_disease,condition
0,TCGA-38-4625-11,TCGA-38-4625,LUAD,66.0,FEMALE,WHITE,Stage IB,NaN,Lung Adenocarcinoma,NaN,...,2973.0,0.0,2973.0,0.0,2973.0,NaN,11.0,Solid Tissue Normal,lung adenocarcinoma,0
1,TCGA-44-6145-11,TCGA-44-6145,LUAD,62.0,FEMALE,WHITE,Stage IA,NaN,Lung Adenocarcinoma,NaN,...,595.0,0.0,595.0,0.0,595.0,NaN,11.0,Solid Tissue Normal,lung adenocarcinoma,0
2,TCGA-50-5930-11,TCGA-50-5930,LUAD,47.0,MALE,WHITE,Stage IIIA,NaN,Lung Adenocarcinoma,NaN,...,282.0,NaN,NaN,1.0,177.0,NaN,11.0,Solid Tissue Normal,lung adenocarcinoma,0
3,TCGA-49-4490-11,TCGA-49-4490,LUAD,45.0,FEMALE,WHITE,Stage IIIA,NaN,Lung Adenocarcinoma,NaN,...,385.0,NaN,NaN,1.0,385.0,NaN,11.0,Solid Tissue Normal,lung adenocarcinoma,0
4,TCGA-55-6970-11,TCGA-55-6970,LUAD,67.0,FEMALE,WHITE,Stage IIIA,NaN,Lung Adenocarcinoma,NaN,...,464.0,NaN,NaN,1.0,457.0,NaN,11.0,Solid Tissue Normal,lung adenocarcinoma,0


In [5]:
expr.head()

sample,sample,LINC00115,C1orf159,TNFRSF4,UBE2J2,DVL1,MXRA8,AURKAIP1,MRPL20,ATAD3C,...,TKTL1,EMD,RPL10,DNASE1L1,PLXNA3,G6PD,GAB3,DKC1,MTCP1,BRCC3
TCGA-38-4625-11,TCGA-38-4625-11,-0.323490,-1.100032,-0.915524,-0.688171,-0.777079,-0.208907,0.873192,-0.610336,-0.001183,...,-1.423766,0.173274,2.271428,-0.290100,-1.877842,0.364662,-1.261048,-2.002651,-0.905306,-1.092115
TCGA-44-6145-11,TCGA-44-6145-11,0.164617,0.191524,-0.435535,0.227018,0.436352,-0.768474,0.333264,0.442010,0.828827,...,0.115755,0.175680,-1.308228,-0.982666,0.499246,-0.378749,-0.287446,1.244888,0.983097,-0.736651
TCGA-50-5930-11,TCGA-50-5930-11,-0.177936,-0.317288,-1.316217,-0.938959,-0.087719,-0.122541,-0.162706,-0.526907,0.083904,...,0.276709,-0.162977,-0.539480,-0.065216,-0.007281,0.140855,0.734071,0.518765,0.561249,-0.318137
TCGA-49-4490-11,TCGA-49-4490-11,-0.590117,-2.212051,-0.891135,-0.450484,-3.015575,-0.932242,-0.306246,1.354675,-1.347085,...,2.823839,-1.232823,2.916698,-1.383080,-2.425153,-2.432705,-2.679173,-1.368930,0.238530,-2.852061
TCGA-55-6970-11,TCGA-55-6970-11,0.140063,-0.588814,0.696697,0.196450,-0.068389,-0.068584,-1.580987,-1.134929,-0.335955,...,0.500780,-0.184624,-0.823877,0.485652,1.173374,0.130703,1.488432,0.932143,0.865795,0.508919


In [6]:
grn.head()

,SYMBOL_TF,SYMBOL_TG
0,PARP1,BRCA2
1,AHR,CYP1B1
2,AHR,FOS
3,AHR,SOS1
4,AHR,UGT1A6


In [11]:
CatCov=['gender']  
ConCov=['age_at_initial_pathologic_diagnosis',]
conCol='condition'

In [12]:
data=run(expression_data=expr,
                   meta=meta, 
                   CatCov=CatCov,
                   ConCov=ConCov,
                   GRN=grn,
                   conCol='condition',
)

Processing edges: 0it [00:00, ?it/s]


ValueError: shapes (50,4) and (3,) not aligned: 4 (dim 1) != 3 (dim 0)

In [9]:
data_old = data.get_results()
data_binary_old = data.get_results_binary()
model_stats_old = data.get_model_stats()

In [10]:
print((data.get_results() != data_old).sum().sum())
print((data.get_results_binary() != data_binary_old).sum().sum())
print((data.get_model_stats() != model_stats_old).sum().sum())

0
0
0
